### Data Cleaning

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# Define the directory where your Parquet files are stored
data_dir = "/Users/HVMS/Desktop/GitHub/CineAI/src/data_preprocessing/Data_Storage/Unstructered Data/parquet"

In [3]:
# Load the Parquet files into DataFrames
details_df = pd.read_parquet(os.path.join(data_dir, "details_df.parquet"))
cast_df = pd.read_parquet(os.path.join(data_dir, "cast_df.parquet"))
crew_df = pd.read_parquet(os.path.join(data_dir, "crew_df.parquet"))
recommendations_df = pd.read_parquet(os.path.join(data_dir, "recommendations_df.parquet"))
watch_providers_df = pd.read_parquet(os.path.join(data_dir, "watch_providers_df.parquet"))

### Inspecting the data

In [8]:
print("Details DF:")
print(details_df.info())
print(details_df.head())

print("\nCast DF:")
print(cast_df.info())
print(cast_df.head())

print("\nCrew DF:")
print(crew_df.info())
print(crew_df.head())

print("\nRecommendations DF:")
print(recommendations_df.info())
print(recommendations_df.head())

print("\nWatch Providers DF:")
print(watch_providers_df.info())
print(watch_providers_df.head())


Details DF:
<class 'pandas.core.frame.DataFrame'>
Index: 234851 entries, 0 to 234851
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   movie_id           234851 non-null  float64       
 1   title              234850 non-null  object        
 2   overview           234850 non-null  object        
 3   release_date       234845 non-null  datetime64[ns]
 4   runtime            234850 non-null  float64       
 5   original_language  234850 non-null  object        
 6   popularity         234850 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 14.3+ MB
None
   movie_id                           title  \
0      98.0                       Gladiator   
1    8871.0  How the Grinch Stole Christmas   
2    7443.0                     Chicken Run   
3    9532.0               Final Destination   
4      77.0                         Memento   

                   

### Handling data types

In [9]:
# Convert release_date to datetime in details_df if not already
if 'release_date' in details_df.columns:
    details_df['release_date'] = pd.to_datetime(details_df['release_date'], errors='coerce')

# Convert IDs to integer type where applicable (handle errors gracefully)
# For example, in details_df movie_id should be int
if 'movie_id' in details_df.columns:
    details_df['movie_id'] = pd.to_numeric(details_df['movie_id'], errors='coerce', downcast='integer')

if 'movie_id' in cast_df.columns:
    cast_df['movie_id'] = pd.to_numeric(cast_df['movie_id'], errors='coerce', downcast='integer')

if 'movie_id' in crew_df.columns:
    crew_df['movie_id'] = pd.to_numeric(crew_df['movie_id'], errors='coerce', downcast='integer')

if 'movie_id' in recommendations_df.columns:
    recommendations_df['movie_id'] = pd.to_numeric(recommendations_df['movie_id'], errors='coerce', downcast='integer')
    # Also convert recommended_movie_id
    recommendations_df['recommended_movie_id'] = pd.to_numeric(recommendations_df['recommended_movie_id'], errors='coerce', downcast='integer')

if 'movie_id' in watch_providers_df.columns:
    watch_providers_df['movie_id'] = pd.to_numeric(watch_providers_df['movie_id'], errors='coerce', downcast='integer')

# Similarly, cast_id, crew_id might need conversion
if 'cast_id' in cast_df.columns:
    cast_df['cast_id'] = pd.to_numeric(cast_df['cast_id'], errors='coerce', downcast='integer')

if 'crew_id' in crew_df.columns:
    crew_df['crew_id'] = pd.to_numeric(crew_df['crew_id'], errors='coerce', downcast='integer')

### Handling Missing Values

In [10]:
# Check missing values
print("\nMissing values in details_df:\n", details_df.isnull().sum())
print("\nMissing values in cast_df:\n", cast_df.isnull().sum())
print("\nMissing values in crew_df:\n", crew_df.isnull().sum())
print("\nMissing values in recommendations_df:\n", recommendations_df.isnull().sum())
print("\nMissing values in watch_providers_df:\n", watch_providers_df.isnull().sum())

# Decide how to handle missing values:
# For now, we might just leave them as-is or handle critical fields only.
# Example: If overview is missing, we can leave it since not all movies need overviews.
# If IDs are missing, consider dropping those rows.
details_df = details_df.dropna(subset=['movie_id'])  # Ensure no movie without ID


Missing values in details_df:
 movie_id             0
title                1
overview             1
release_date         6
runtime              1
original_language    1
popularity           1
dtype: int64

Missing values in cast_df:
 movie_id     0
cast_id      0
name         0
character    0
credit_id    0
dtype: int64

Missing values in crew_df:
 movie_id      0
crew_id       0
name          0
department    0
job           0
credit_id     0
dtype: int64

Missing values in recommendations_df:
 movie_id                   0
recommended_movie_id       0
recommended_title          0
recommended_popularity    47
recommended_genre_ids     47
dtype: int64

Missing values in watch_providers_df:
 movie_id         0
provider_type    0
provider_name    0
link             0
dtype: int64


In [13]:
# Convert recommended_genre_ids from list/array to tuple
recommendations_df['recommended_genre_ids'] = recommendations_df['recommended_genre_ids'].apply(
    lambda x: tuple(x) if isinstance(x, (list, np.ndarray)) else x
)

# Now try checking duplicates again
print("Duplicates in recommendations_df:", recommendations_df.duplicated().sum())

Duplicates in recommendations_df: 61040


### Handling Duplicates

In [ ]:
# Checking for duplicates
print("\nDuplicates in details_df:", details_df.duplicated().sum())
print("Duplicates in cast_df:", cast_df.duplicated().sum())
print("Duplicates in crew_df:", crew_df.duplicated().sum())
print("Duplicates in recommendations_df:", recommendations_df.duplicated().sum())
print("Duplicates in watch_providers_df:", watch_providers_df.duplicated().sum())


Duplicates in details_df: 4265
Duplicates in cast_df: 52792
Duplicates in crew_df: 72120
Duplicates in recommendations_df: 61040
Duplicates in watch_providers_df: 11709


In [ ]:
# Removing Duplicates if had
details_df.drop_duplicates(inplace=True)
cast_df.drop_duplicates(inplace=True)
crew_df.drop_duplicates(inplace=True)
recommendations_df.drop_duplicates(inplace=True)
watch_providers_df.drop_duplicates(inplace=True)

In [16]:
# Checking again for duplicates
print("\nDuplicates in details_df:", details_df.duplicated().sum())
print("Duplicates in cast_df:", cast_df.duplicated().sum())
print("Duplicates in crew_df:", crew_df.duplicated().sum())
print("Duplicates in recommendations_df:", recommendations_df.duplicated().sum())
print("Duplicates in watch_providers_df:", watch_providers_df.duplicated().sum())


Duplicates in details_df: 0
Duplicates in cast_df: 0
Duplicates in crew_df: 0
Duplicates in recommendations_df: 0
Duplicates in watch_providers_df: 0


### Normalization and Merging

In [17]:
clean_data_dir = "/Users/HVMS/Desktop/GitHub/CineAI/src/data_preprocessing/Data_Storage/Structured Data/parquet"
os.makedirs(clean_data_dir, exist_ok=True)

details_df.to_parquet(os.path.join(clean_data_dir, "details_df_clean.parquet"), index=False)
cast_df.to_parquet(os.path.join(clean_data_dir, "cast_df_clean.parquet"), index=False)
crew_df.to_parquet(os.path.join(clean_data_dir, "crew_df_clean.parquet"), index=False)
recommendations_df.to_parquet(os.path.join(clean_data_dir, "recommendations_df_clean.parquet"), index=False)
watch_providers_df.to_parquet(os.path.join(clean_data_dir, "watch_providers_df_clean.parquet"), index=False)

print("\nData cleaning and normalization completed. Cleaned files saved to:", clean_data_dir)


Data cleaning and normalization completed. Cleaned files saved to: /Users/HVMS/Desktop/GitHub/CineAI/src/data_preprocessing/Data_Storage/Structured Data/parquet
